In [ ]:
%matplotlib notebook

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.stats as stats
from design.spectral import Spectral
np.random.seed(0)

multiplier = 1
plot = 1 # 1: filled contour, 2: just level curves, else: greyscale image
initialization = 'random' #'random', 'separated_x', 'separated_y', 'trivial_x', 'trivial_y', 
Lx = Ly = 1
Nx = 100
Ny = 100
# Nx = 128
# Ny = 64
eps = multiplier*0.03253176201497791
dt, Nt = 0.0001, 10
show_error=False

spec = Spectral(Lx, Ly, Nx, Ny, eps, initialization)

# |k|^4 plot:

if plot==1:
    plt.contourf(spec.k2**2, cmap=plt.cm.gray)
elif plot==2:
    plt.contour(spec.k2**2, cmap=plt.cm.gray)
else:
    plt.imshow(spec.k2**2, cmap=plt.cm.gray, origin='lower')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].set_title("Visualization")
axs[1].set_title("Histogram of relative concentrations")

plot=plot
bins=levels=32

if(plot==1):
    axs[0].contourf(spec.u, levels=levels, cmap=plt.cm.gray)
if(plot==2):
    axs[0].contour(spec.u, levels=levels, cmap=plt.cm.gray)
else:
    image = axs[0].imshow(spec.u, cmap=plt.cm.gray, origin='lower', animated=True)

if show_error:
    error_info = axs[0].text(0.5,0.85, f"Difference: {0}",
        bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
        transform=axs[0].transAxes, ha="center")

histogram = axs[1].hist(spec.u.flatten(), bins=bins)

def animate(i):
    spec.advance(dt, Nt)
    if(plot==1):
        axs[0].clear()
        axs[0].contourf(spec.u, levels=levels, cmap=plt.cm.gray)
    if(plot==2):
        axs[0].clear()
        axs[0].contour(spec.u, levels=levels, cmap=plt.cm.gray)
    else:
        image.set_data(spec.u)
    axs[1].cla()
    distribution = spec.u.flatten()

    histogram = axs[1].hist(distribution, bins=bins)
    if show_error:
#         err, = [np.log(np.abs(np.sum(spec.u)/(Nx*Ny)/deviation - 1))/np.log(10)
        err, = [(np.abs(np.sum(spec.u)/(Nx*Ny) - deviation))
        if deviation!=0 else -np.inf]
        error_info.set_text(f"Log-error: {err}")
    return 

anim = animation.FuncAnimation(fig=fig, func=animate)

plt.show()

In [ ]:
"""Keep pressing Ctrl+Return on this cell to monitor the satate of the concentrations"""
temp = f"Total: {spec.u.size}", f"Below viable: {spec.u[spec.u>1].size}", f"Above viable: {spec.u[spec.u<-1].size}"
print(temp)
spec.u.size, np.sum(spec.u>1), np.sum(spec.u<-1)